# Programming Assignment 1: Convolution and Back-Propagation

**UBC CPEN 400D: Deep Learning, 2022 Winter Term 2**

**Created By Renjie Liao**

**Date: Jan. 24, 2023**

---
# Setup

We will use PyTorch to implement this assignment.

In [1]:
# Imports
import pdb
import torch
import numpy as np
from torch import nn
from math import pi
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets, transforms, utils

torch.set_default_dtype(torch.float64)

## load MNIST images
B = 5 # batch size
train_set = datasets.MNIST('./data',
                            train=True,
                            download=True,
                            transform=transforms.ToTensor())

loader = torch.utils.data.DataLoader(train_set, batch_size=B)

## load a batch of MNIST images as a PyTorch tensor (shape: B x C x H x W)
# B: batch size
# C: number of channels
# H: height of images
# W: width of images
img, label = next(iter(loader)) # img shape: B x C x H x W, label shape: B X 1

## create a random filter (shape: D x C x K x K)
K = 3 # kernel size
P = 1 # padding size
C = 1 # channel size
D = 2 # number of filters
filter = torch.randn(D, C, K, K) # filter shape: D x C x K x K

In [2]:
# pad_img_1_1 = np.pad(img_1_1, ((p,p),(p,p)), 'constant', constant_values = 0)
def padding_scratch(img, padding):
  h, w = img.shape
  h_pad, w_pad = h+2*padding, w+2*padding
  padded_img = torch.zeros((h_pad, w_pad))
  for i in range(h):
    for j in range(w):
      padded_img[padding+i][padding+j] = img[i][j]
  return padded_img


In [3]:
#padded_img = np.pad(img, ((0,0), (0,0),(pad,pad), (pad,pad)), 'constant', constant_values = (0,0))
def padding_scratch4(img, pad):
  b,c,h,w = img.shape
  h_pad, w_pad = h+2*pad, w+2*P
  # print(b,c,h,w)
  padded_img = torch.zeros((b,c,h_pad, w_pad))
  for i in range(b):
    for j in range(c):
      for k in range(h):
        for l in range(w):
          padded_img[i][j][pad+k][pad+l] = img[i][j][k][l]
  return padded_img


In [4]:
#np.pad(h, ((hp),(hp)), 'constant', constant_values = 0) #need it for cyclic_matrix column #(2wh - 
def padding_scratch_vec(vec, pad):
  print(vec.shape)
  h = vec.shape[0]
  h_pad = h+2*pad
  padded_vector = torch.zeros((h_pad,))
  for i in range(h):
    padded_vector[pad+i] = vec[i]
  return padded_vector


In [5]:
# img = np.ones((2,3))
# padded_img = padding_scratch(img, 2)
# print(padded_img)

In [6]:
# #remove before submit it.
# print("the img shape is ", img.shape)
# print("filter shape is", filter.shape)

In [7]:
# img_0 = img[0]
# print(img_0.shape)
# img_1 = img[1]
# print(img_1.shape)
# img_2 = img[2]
# print(img_2.shape)
# img_1_1 = img_1[0]
# print(img_1_1.shape)
# img_1_1_1 = img_1_1[0]
# print(img_1_1_1.shape)

In [8]:
# filter_1 = filter[1]
# print(filter_1.shape)
# filter_1_1 = filter_1[0]
# print(filter_1_1.shape)
# print(filter_1_1)
# vectorized_1d_filter = filter_1_1.flatten() #1x(KxK)
# print(vectorized_1d_filter.shape)
# print(vectorized_1d_filter)

In [9]:
# print(img)

In [10]:
# B, C, H, W = img.shape
# print(B)
# print(C)
# print(H)
# print(W)

In [11]:
# output_w = (W+2*1 - K)/1 + 1
# output_h = (H+2*1 - K)/1 + 1

# print(output_w, output_h)

In [12]:
# k = 3
# s = 1
# for i in range(int(output_w)):
#   for j in range(int(output_h)):
#     print(0+s*i, k+s*i)
#     print(0+s*j, k+s*j)

In [13]:
# sum_matrix = np.zeros((4,4))
# result_matrix = np.ones((4,4))
# print(sum_matrix)
# print(result_matrix)
# final_matrix = sum_matrix + result_matrix
# print(final_matrix)

# #slicing
# mid_matrix = final_matrix[0:2, 0:2]
# print(mid_matrix.shape)
# print(mid_matrix)

In [14]:
# p = 1

In [15]:
# pad_img_1_1 = np.pad(final_matrix, ((p,p),(p,p)), 'constant', constant_values = 0)
# print(pad_img_1_1.shape)
# print(pad_img_1_1)

---
# Q1 [60Pts]: 2D convolution and its gradient




## 1.1 [30Pts]  Implement 2D convolution:

Discrete convolution can be implemented in multiple ways, e.g., matrix multiplication in spatial/Fourier domains.

First, let us take a look at 1D convolution in spatial domain. Suppose we have a 1D signal with $n$ elements $x_1, x_2, \dots, x_n$ and a 1D filter with $m$ weights $h_1, h_2, \dots, h_m$. Note that we typically use filters with odd sizes for the ease of indexing.

The (discrete) convolution with zero-padding and stride 1 is defined as:

\begin{align}
    y = h \ast x = \sum_{i=1}^{n} \sum_{j=1}^{m} h_j x_{i - \lfloor m/2 \rfloor - 1 + j},
\end{align}
where padded values $x_{-\lfloor m/2 \rfloor + 1}, \dots, x_{0}, x_{n+1}, \dots, x_{n - \lfloor m/2 \rfloor - 1 + m}$ are all zeros.

If you forget about the concepts of padding and stride, take a look at [this guide](https://arxiv.org/pdf/1603.07285.pdf) or [these pictures](https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md).

In the 1D case, we can illustrate the two matrix multiplication views of spatial convolution as below.

1.   **im2col**: 
The key idea is to first **extract the spatial windows from the signal** $x$ for individual convolutions and then perform convolutions (i.e., dot product with the filter).
If we put each window as a column in a matrix (the right one in RHS below), then we can perform convolution via the following matrix multiplication (N.B.: the products between the filter and individual columns can be done in parallel).

\begin{align}
    y^\top = (h \ast x)^{\top} = \begin{bmatrix}
                h_m & h_{m-1} & \cdots & h_3 & h_2 & h_1
            \end{bmatrix}
            \begin{bmatrix}
                x_{m - \lfloor m/2 \rfloor} & x_{m - \lfloor m/2 \rfloor + 1} & \cdots & x_m & x_{m+1} & \cdots & 0 & 0 \\
                \vdots & \vdots & \cdots & x_{m-1} & x_m & \cdots & \vdots & \vdots \\
                x_1 & x_2 & \cdots & \vdots & x_{m-1} & \cdots  & x_n & 0 \\
                0 & x_1 & \cdots & \vdots & \vdots & \cdots  & x_{n-1} & x_n \\
                \vdots & 0 & \cdots & \vdots & \vdots & \cdots & \vdots & \vdots \\                        
                0 & 0 & \cdots & x_1 & x_2 & \cdots & x_{n - \lfloor m/2 \rfloor+1} & x_{n - \lfloor m/2 \rfloor}
            \end{bmatrix}.
\end{align}

2.   **filter2row**: The key idea is to convert the filter and the signal to a sparse cyclic matrix and a vector respectively.
Then the convolution is simply the matrix multiplication between the filter and the signal.

\begin{align}
        y = h \ast x = 
            \begin{bmatrix}
                h_{\lfloor m/2 \rfloor + 1} & h_{\lfloor m/2 \rfloor + 2} & \cdots & h_m & 0 & \cdots & \cdots & \cdots & \cdots & 0 \\
                h_{\lfloor m/2 \rfloor} & h_{\lfloor m/2 \rfloor + 1} & \cdots & h_{m-1} & h_m & 0 & \cdots & \cdots & \cdots & 0 \\
                \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
                h_1 & h_2 & \cdots & \cdots & \cdots & \cdots & h_m & 0 & \cdots & 0 \\
                0 & h_1 & h_2 & \cdots & \cdots & \cdots & \cdots & h_m & \cdots & 0 \\
                \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
                0 & 0 & \cdots & \cdots & \cdots & 0 & h_1 & h_2 & \cdots & h_{\lfloor m/2 \rfloor + 2} \\                
                0 & 0 & \cdots & \cdots & & \cdots \cdots & 0 & h_1 & \cdots & h_{\lfloor m/2 \rfloor + 1}
            \end{bmatrix}
            \begin{bmatrix}
                x_1 \\
                x_2 \\
                x_3 \\
                \vdots \\
                x_n
            \end{bmatrix}
\end{align}

**Task**: 
Implement the 2D convolution in the spatial domain via matrix multiplication following the above two views: **im2col** and **filter2row**.
The starter code is provided below.
You just need to fill in the missing parts of function ***conv2d_im2col*** and ***conv2d_filter2row***.
If your implementation is correct, the ***unit_test*** will output:

*Your implementation of xxx is correct!*

Otherwise, it will output:

*Your implementation of xxx is wrong!*

**N.B.**: we assume the strides along height and width are the same and the kernel is square

**Hint**: you can reduce the 2D case to 1D and follow the above construction.

In [16]:
# vector = np.zeros((9,))
# print(vector)
# padd = 2
# padd_vector = np.pad(vector, ((padd),(padd)), 'constant', constant_values = 1)
# print(padd_vector.shape)
# print(padd_vector)

In [17]:
# padd_vector[0] = 2
# print(padd_vector)
# # print(padd_vector.reverse()) #'numpy.ndarray' object has no attribute 'reverse'
# #brr[:] = brr[::-1]
# reverse = padd_vector[::-1]
# print(reverse)

In [18]:
# matrix = np.zeros((3,3))
# print(matrix)
# # matrix[0:1, 1] = 1 #(0,1)
# # print(matrix)
# for i in range(3):
#   matrix[i:i+1, 1] = 1 #column[1] = [1,1,1] 
# print(matrix)

# print("hi")
# print(matrix[0, :])
# print(matrix[0, :].shape)

# print(vector[0:3].shape)
# print(vector[0:3])

# # cyclic_matrix[jeno:jeno+1,yj] = padded_h[0+yj:yj + (k*k)//2]

In [19]:
# matrix = np.zeros((3,3))
# print(matrix)
# vector = [1,2,3]
# matrix[:, 0] = vector
# print(matrix)
# # reverse = slide[::-1]
# reverse = vector[::-1]
# matrix[:, 1] = reverse
# print(matrix)

In [20]:
## implement the following two functions
def conv2d_im2col(img, filter, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    ### Fill in this function ###
    # Args:
    #   img: images, shape B x C x H x W
    #   filter: filters, shape D x C x K x K
    #   channel_size: number of channels, scalar (C)
    #   num_filters: number of filters, scalar (D)
    #   kernel_size: kernel size, scalar (K)
    #   stride: stride size, scalar
    #   padding: padding size, scalar
    #
    # Returns:
    #   out: convoluted images, shape B x D x H x W

    #get input shape
    B, C, H, W = img.shape
    #get filter shape
    D, C, K, K = filter.shape
    D = num_filters ##
    C = channel_size ##
    K = kernel_size ##

    #get stride, padding
    s = stride
    p = padding

    #shape of output w', h'
    output_w = (W+2*p - K)/s + 1
    output_h = (H+2*p - K)/s + 1

    #(D, h', w')
    d3_tensor = torch.zeros((D, int(output_h), int(output_w)))

    #(B, D, h', w')
    d4_tensor = torch.zeros((B, D, int(output_h), int(output_w)))

    # #do padding first
    # np.pad(bb, ((2,2),(2,2)), 'constant', constant_values=0)
    for i in range(B): #batch imgs -> 1 img
      img_1 = img[i] #img_1 shape:  C x H x W
      
      for j in range(D): #D filters -> 1 filter
        filter_1 = filter[j] #filter_1 shape: C x K x K
        
        for k in range(C): #C channels -> 1 channel (2d convolution 2d)
          img_1_1 = img_1[k] #img_1_1 shape: H x W
          filter_1_1 = filter_1[k] #filter_1_1 shape: K x K
          #padding
          # pad_img_1_1 = np.pad(img_1_1, ((p,p),(p,p)), 'constant', constant_values = 0)
          pad_img_1_1 = padding_scratch(img_1_1, p)
          sum_matrix = torch.zeros((int(output_h), int(output_w)))
          result_matrix = torch.zeros((int(output_h), int(output_w)))

          """
          result_matrix = ~~~
          sum_matrix += result_matrix (for loop로 element-wise addition해줘야 하나?)
          """
          vectorized_1d_filter = filter_1_1.flatten() #1x(KxK)
          img2_col = torch.zeros((K*K, int(output_w*output_h))) #(kxk)x(w'xh')

          for yj in range(int(output_h)):
            for jeno in range(int(output_w)):
              window = pad_img_1_1[0+s*yj:K+s*yj, 0+s*jeno:K+s*jeno]
              #print("window shape is ", window.shape)
              flat_window = window.flatten()
              #print("flat window shape is ", flat_window.shape)  #column vector (KxK)x1
              
              # column_window = np.transpose(flat_window) #column vector (KxK)x1
              # print("column window shape is", column_window.shape)
              img2_col[:, int(yj*output_h + jeno)] = flat_window #0~output_w*output_h-1
          
          # for yj in range(output_w*output_h): #slide = output_w*output_h 개
          #     print("hello")
          #     window = pad_img_1_1[0:k,0:k] #slicing. window shape: kxk #need to be changed
          #     flat_window = window.flatten()
          #     column_window = np.transpose(flat_window) #column vector. (KxK)x1
          #     img2_col[:, yj] = column_window
          
          #print(vectorized_1d_filter.shape)
          #print(img2_col.shape)
          result_matrix = torch.matmul(vectorized_1d_filter, img2_col)
          #result_matrix = torch.dot(vectorized_1d_filter, img2_col)
          #print(result_matrix.shape) #(output_w)x(output_h)
          result_matrix = torch.reshape(result_matrix, (int(output_w), int(output_h)))
          sum_matrix = sum_matrix + result_matrix #sum_matrix: (output_w)x(output_h)

          # print("hi")
          # print("sum_matrix.shape is ", sum_matrix.shape)
      #다시 up dimension해줘야.
        d3_tensor[j] = sum_matrix
   #다시 up dimension해줘야. 
      d4_tensor[i] = d3_tensor
    return d4_tensor #torch.tensor(d4_tensor)
    #return output # convoluted images, shape B x D x H x W
    # pass


def conv2d_filter2row(img, filter, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    ### Fill in this function ###
    # Args:
    #   img: images, shape B x C x H x W
    #   filter: filters, shape D x C x K x K
    #   channel_size: number of channels, scalar (C)
    #   num_filters: number of filters, scalar (D)
    #   kernel_size: kernel size, scalar (K)
    #   stride: stride size, scalar
    #   padding: padding size, scalar
    #get input shape
    B, C, H, W = img.shape
    #get filter shape
    D, C, K, K = filter.shape
    D = num_filters ##
    C = channel_size ##
    K = kernel_size ##

    #get stride, padding
    s = stride
    p = padding

    #shape of output w', h'
    output_w = (W+2*p - K)/s + 1
    output_h = (H+2*p - K)/s + 1

    #(D, h', w')
    d3_tensor = torch.zeros((D, int(output_w), int(output_h)))

    #(B, D, h', w')
    d4_tensor = torch.zeros((B, D, int(output_w), int(output_h)))

    # #do padding first
    # np.pad(bb, ((2,2),(2,2)), 'constant', constant_values=0)
  #   for i in range(B): #batch imgs -> 1 img
  #     img_1 = img[i] #img_1 shape:  C x H x W
      
  #     for j in range(D): #D filters -> 1 filter
  #       filter_1 = filter[j] #filter_1 shape: C x K x K
        
  #       for k in range(C): #C channels -> 1 channel (2d convolution 2d)
  #         img_1_1 = img_1[k] #img_1_1 shape: H x W
  #         filter_1_1 = filter_1[k] #filter_1_1 shape: K x K
  #         #padding
  #         #pad_img_1_1 = np.pad(img_1_1, ((p,p),(p,p)), 'constant', constant_values = 0)
  #         #print("pad_img_1_1 shape is ", pad_img_1_1.shape) #30 x 30
  #         sum_matrix = torch.zeros((int(output_w), int(output_h)))
  #         result_matrix = torch.zeros((int(output_w), int(output_h)))

  #         cyclic_matrix = torch.zeros((int(output_w*output_h), int(output_w*output_h))) #hw * hw
  #         vectorized_img = img_1_1.flatten() #hw*1 = 784
  #         #print(vectorized_img.shape)
  #         h = filter_1_1.flatten() #(KxK)x1. h = [h1, h2, ... h[k^2/2]+1, hk^2]
  #         #print("h shape is ",h.shape)
  #         hp = int(output_w*output_h - (K*K//2) -1) #of padding
  #         #np.pad
  #         padded_h = padding_scratch_vec(h, hp) #np.pad(h, ((hp),(hp)), 'constant', constant_values = 0) #need it for cyclic_matrix column #(2wh - 1)x1
  #         #print("padded_h shape is ", padded_h.shape) #2*w*h - 1. (1567, )
          
  #         #for loop. make cyclic_matrix
  #         for yj in range(int(output_w*output_h)): #yj: column index. 0~(wh-1)
  #           # for jeno in range(int(output_w*output_h)): #jeno: row index. 0~(wh-1)
  #           # print(cyclic_matrix[:,yj].shape)
  #           # print(padded_h[(0+yj):(yj + int(output_h*output_w))].shape)
  #           slide = padded_h[(0+yj):(yj + int(output_h*output_w))]
  #           #print("slide shape is", slide.shape)
            
  #           # print("0+yj is ", 0+yj)
  #           # print("yj + int(output_h*output_w is ", yj + int(output_h*output_w))
  #           #brr[:] = brr[::-1]

  #           reverse = slide[::-1]
            
  #           cyclic_matrix[:,yj] = reverse
  #           # print(matrix[0, :].shape)
  #           # print(vector[0:3].shape)

  #         # print("cyclic_matrix[:, 0]", cyclic_matrix[:, 0])
  #         # print("cyclic_matrix[:, -1]", cyclic_matrix[:, -1])
  #         break
  #         result_matrix = torch.matmul(cyclic_matrix, vectorized_img)
  #         # print("result_matrix.shape is ", result_matrix.shape) #hw x 1

  #         #print(result_matrix.shape) #(output_w)x(output_h)
  #         result_matrix = torch.reshape(result_matrix, (int(output_w), int(output_h)))
  #         sum_matrix = sum_matrix + result_matrix #sum_matrix: (output_w)x(output_h)
          
  #         #print("hi")
  #         #print("sum_matrix.shape is ", sum_matrix.shape)
  #     #다시 up dimension해줘야.
  #       d3_tensor[j] = sum_matrix
  #  #다시 up dimension해줘야. 
  #     d4_tensor[i] = d3_tensor
    return d4_tensor#torch.tensor(d4_tensor)

    # Returns:
    #   out: convoluted images, shape B x D x H x W
    pass


def unit_test_conv2d(img, filter, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    # call your implemented "im2col" conv2D
    y_im2col = conv2d_im2col(img, filter, channel_size=channel_size, num_filters=num_filters, kernel_size=kernel_size, stride=stride, padding=padding)

    # ground truth conv2D
    y_gt = F.conv2d(img, filter, stride=stride, padding=padding)
    
    diff = (y_im2col - y_gt).norm()    
    if diff < 1.0e-5:
        print("Your implementation of conv2d_im2col is correct!")
    else:
        print("Your implementation of conv2d_im2col is wrong!")
    
    # call your implemented "im2col" conv2D
    y_filter2row = conv2d_filter2row(img, filter, channel_size=channel_size, num_filters=num_filters, kernel_size=kernel_size, stride=stride, padding=padding)
    
    diff = (y_filter2row - y_gt).norm()
    if diff < 1.0e-5:
        print("Your implementation of conv2d_filter2row is correct!")
    else:
        print("Your implementation of conv2d_filter2row is wrong!")


unit_test_conv2d(img, filter, channel_size=C, num_filters=D, kernel_size=K, stride=1, padding=P)

Your implementation of conv2d_im2col is correct!
Your implementation of conv2d_filter2row is wrong!


In [21]:
# a = np.zeros((1,))
# a

## 1.2 [20Pts] Implement the gradient of 2D convolution

We now turn to the gradient of 2D convolution.
In particular, given a batch of images $x$ with the shape $B \times C \times H \times W$ and filters $h$ with shape $D \times C \times K \times K$, we can view the convolution (zero-padding and stride 1) as a function
\begin{align}
    y = f(h, x),
\end{align}
that would produce an output tensor $y$ with shape $B \times D \times H \times W$.

If we vectorize $x$, $h$, and $y$, then the Jacobian matrix $∇f = [\frac{\partial y}{\partial h}, \frac{\partial y}{\partial x}]$ would be of shape $BDHW \times (DCKK + BCHW)$.
In practice, we almost never need to compute the Jacobian matrix directly as it is unnecessary for back-propagation.
Instead, we often need to compute the product between the transposed Jacobian and a vector (a.k.a. vector-Jacobian product), i.e., ${\frac{\partial y}{\partial h}}^{\top} v$ and ${\frac{\partial y}{\partial x}}^{\top} v$.
For example, the vector $v$ could the gradient of some loss $\ell$ (scalar) w.r.t. the output above (i.e., $\frac{\partial \ell}{\partial y}$).

**Task**: 
Given input images $x$, filters $h$, output $y$, and a vector $v$, implement the gradients ${\frac{\partial y}{\partial h}}^{\top} v$ and ${\frac{\partial y}{\partial x}}^{\top} v$ in the function below.

**N.B.**: The function needs to return the gradients in the original shapes, i.e., ${\frac{\partial y}{\partial h}}^{\top} v$ should have the same shape as $h$ ($D \times C \times K \times K$) and ${\frac{\partial y}{\partial x}}^{\top} v$ should have the same shape as $x$ ($B \times C \times H \times W$).

In [22]:
# a = np.zeros((2,3,4,5))
# print(a.shape)
# b = np.zeros((2,2,3))
# b[0, 1, 2] = 1
# print(b)

# a_vector = a.flatten()
# print(a_vector.shape)
# b_vector = b.flatten()
# print(b_vector.shape)
# print(b_vector)

In [23]:
# a = np.ones((2,3,4,5))
# pad = 1
# a_pad = np.pad(a, ((0,0), (pad,pad), (pad,pad), (0,0)), 'constant', constant_values = (0,0))
# print(X_pad.shape)

In [24]:
## implement the following functions
def grad_conv2d(img, filter, out, grad_out, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    ### Fill in this function ###
    # Args:
    #   img: images, shape B x C x H x W
    #   filter: filters, shape D x C x K x K
    #   out: convoluted images, shape B x D x H x W
    #   grad_out: gradient w.r.t. output, shape B x D x H x W (e.g gradient of loss w.r.t. output of conv layer) = v

    #   channel_size: number of channels, scalar (C)
    #   num_filters: number of filters, scalar (D)
    #   kernel_size: kernel size, scalar (K)
    #   stride: stride size, scalar
    #   padding: padding size, scalar

    B, C, H, W = img.shape
    D, C, K, K = filter.shape
    s = stride
    pad = padding

    #print("B, C, H, W, D is ", B, C, H, W, D)

    #tensor to numpy
    img = img #.detach().numpy()
    filter = filter #.detach().numpy()
    grad_out = grad_out #.detach().numpy()

    grad_img = torch.zeros((B, C, H, W))
    grad_filter = torch.zeros((D,C,K,K))

    '''
    convolution(not img2col, filter2row)
    convolution between sliding window(KxK size)&filter
    '''
    #padded_img = np.pad(img, ((0,0), (0,0),(pad,pad), (pad,pad)), 'constant', constant_values = (0,0))
    padded_img = padding_scratch4(img, pad) 
    #print(padded_img.shape)
    #padded_grad_img = np.pad(grad_img, ((0,0), (0,0),(pad,pad), (pad,pad)), 'constant', constant_values = (0,0))
    padded_grad_img = padding_scratch4(grad_img, pad)
    #print("padded_img shape",padded_img.shape)
    
    for i in range(B): #batchsize
      #ith img from img, grad_img
      pad_img_1 = padded_img[i] #(C, H+2p, W+2p)
      pad_grad_img_1 = padded_grad_img[i] #(C, H+2p, W+2p)
      for h in range(H): #height
        for w in range(W): #width
          for d in range(D): #output img's channel (#filters)
            
            #window (convolution w/ filter)
            window_1 = pad_img_1[:,h:h+K, w:w+K]

            #update gradients for the window and the filter
            pad_grad_img_1[:, h:h+K, w:w+K] += filter[d,:,:,:] * grad_out[i,d,h,w]
            # print(i, d, h, w)
            # print(grad_filter[d, :, :, :])
            # print(grad_out[i,d,h,w])
            grad_filter[d,:, :, :]+= window_1*grad_out[i,d,h,w]
      grad_img[i, :, :, :] = pad_grad_img_1[:, pad:-pad, pad:-pad]

    # Returns:
    #   grad_img: gradient w.r.t. img, shape B x C x H x W
    #   grad_filer: gradient w.r.t. filter, shape D x C x K x K
    return torch.tensor(grad_img), torch.tensor(grad_filter)
    #pass


def unit_test_grad_conv2d(img, filter, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    filter.requires_grad = True
    img.requires_grad = True

    ### ground truth conv2D
    img_out = F.conv2d(img, filter, stride=stride, padding=padding)

    # create a random vector v
    v = torch.randn_like(img_out)

    # call your implemented "grad_conv2d" function
    grad_img, grad_filter = grad_conv2d(img, filter, img_out, v, channel_size=channel_size, num_filters=num_filters, kernel_size=kernel_size, stride=stride, padding=padding)

    # compute ground-truth gradients
    grad_img_gt = torch.autograd.grad(img_out, img, grad_outputs=v, retain_graph=True)[0]
    grad_filter_gt = torch.autograd.grad(img_out, filter, grad_outputs=v, retain_graph=True)[0]
    #pdb.set_trace()

    diff = (grad_img - grad_img_gt).norm()    
    if diff < 1.0e-5:
        print("Your implementation of grad_img is correct!")
    else:
        print("Your implementation of grad_img is wrong!")

    diff = (grad_filter - grad_filter_gt).norm()    
    if diff < 1.0e-5:
        print("Your implementation of grad_filter is correct!")
    else:
        print("Your implementation of grad_filter is wrong!")        

unit_test_grad_conv2d(img, filter, channel_size=C, num_filters=D, kernel_size=K, stride=1, padding=P)

<ipython-input-24-cb11f016126d>:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(grad_img), torch.tensor(grad_filter)


Your implementation of grad_img is correct!
Your implementation of grad_filter is correct!


---
## 1.3 [10Pts]: Implement gradient checking via the finite difference approximation

We verify the correctness of the implementation of gradient operators by calling PyTorch's autograd function.
However, PyTorch's autograd function just calls the gradient operators implemented by the PyTorch team.
How do they verify the correctness of their implementation?

The answer is **finite difference approximation**.
Following the setup in 1.2, given a batch of images $x$ with the shape $B \times C \times H \times W$ and filters $h$ with shape $D \times C \times K \times K$, we have the convolution (zero-padding and stride 1)
\begin{align}
    y = f(h, x).
\end{align}

Again, mentally vectorize $x$ and $y$ would help us understand the math. 
Given any vector $v$ with the same shape as $y$, we are interested in computing ${\frac{\partial y}{\partial h}}^{\top} v$ and ${\frac{\partial y}{\partial x}}^{\top} v$. 
These two gradients are equivalent to ${\frac{\partial \ell}{\partial h}}$ and ${\frac{\partial \ell}{\partial x}}$ where
\begin{align}
    \ell(h, x) = y^{\top}v = f(h, x)^{\top} v.
\end{align}
Note that here $\ell$ becomes a scalar.
Based on Talyor's theorem, we have
\begin{align}
    d^{\top} \frac{\partial \ell}{\partial h} = \lim_{\epsilon → 0} \frac{\ell(h + \epsilon \cdot d, x) - \ell(h - \epsilon \cdot d, x)}{2 ϵ},
\end{align}
where $d$ could be any direction vector and $ϵ$ is a scalar.
For our purpose, we just need to set $d$ to be the unit vector to compute the per-dimension value of $\frac{\partial \ell}{\partial h}$. 
Specifically, if we set $d$ as the $i$-th unit vector $e_i$, i.e., $d[i] = 1$ and $d[j] = 0, \forall j \neq i$, we can then compute 
\begin{align}
    \frac{\partial \ell}{\partial h}[i] &= \lim_{\epsilon → 0} \frac{\ell(h + \epsilon \cdot e_i, x) - \ell(h - \epsilon \cdot e_i, x)}{2 ϵ} \\
    & ≈ \frac{\ell(h + \epsilon \cdot e_i, x) - \ell(h - \epsilon \cdot e_i, x)}{2 ϵ}.
\end{align}

**Task**: Implement the finite-difference based gradient checker for ${\frac{\partial \ell}{\partial h}}$ and ${\frac{\partial \ell}{\partial x}}$. 


**N.B.**: For efficiency consideration in the unit test, you can use F.conv2d to compute the convolution in your implementation of *grad_checker*. This assignment is to let you understand how to implement finte-difference. But in pratice, if we want to verify our implementation of conv2d, then we should use our conv2d instead of F.conv2d from PyTorch.


In [25]:
## implement the following functions
def grad_checker(img, filter, conv, grad_out, epsilon=1.0e-5, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    ### Fill in this function ###
    # Args:
    #   img: images, shape B x C x H x W
    #   filter: filters, shape D x C x K x K
    #   conv: convolution function
    #   out: convoluted images, shape B x D x H x W (can get using, img, filter, conv)
    #   grad_out: gradient w.r.t. output, shape B x D x H x W (loss w.r.t out)
    #   channel_size: number of channels, scalar (C)
    #   num_filters: number of filters, scalar (D)
    #   kernel_size: kernel size, scalar (K)
    #   stride: stride size, scalar
    #   padding: padding size, scalar
    #
    B,C,H,W = img.shape
    D,C,K,K = filter.shape

    # for i in range(D*C*K*K):
    #   ei = torch.zeros((D*C*K*K))



    
    grad_img = torch.zeros(B,C,H,W)
    grad_filter = torch.zeros(D,C,K,K)
    # Returns:
    #   grad_img: gradient w.r.t. img, shape B x C x H x W
    #   grad_filer: gradient w.r.t. filter, shape D x C x K x K
    return grad_img, grad_filter
    # pass


def unit_test_grad_checker(img, filter, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    epsilon = 1.0e-5
    filter.requires_grad = True
    img.requires_grad = True

    ### ground truth conv2D
    img_out = F.conv2d(img, filter, stride=stride, padding=padding)

    # create a random vector v
    v = torch.randn_like(img_out)

    # call your implemented "grad_checker" function
    grad_img, grad_filter = grad_checker(img, filter, F.conv2d, v, epsilon=epsilon, channel_size=channel_size, num_filters=num_filters, kernel_size=kernel_size, stride=stride, padding=padding)

    # compute ground-truth gradients
    grad_img_gt = torch.autograd.grad(img_out, img, grad_outputs=v, retain_graph=True)[0]
    grad_filter_gt = torch.autograd.grad(img_out, filter, grad_outputs=v, retain_graph=True)[0]
    #pdb.set_trace()

    diff = (grad_img - grad_img_gt).norm()    
    if diff < 1.0e-5:
        print("Your implementation of grad_img is correct!")
    else:
        print("Your implementation of grad_img is wrong!")

    diff = (grad_filter - grad_filter_gt).norm()    
    if diff < 1.0e-5:
        print("Your implementation of grad_filter is correct!")
    else:
        print("Your implementation of grad_filter is wrong!")        

unit_test_grad_checker(img, filter, channel_size=C, num_filters=D, kernel_size=K, stride=1, padding=P)

Your implementation of grad_img is wrong!
Your implementation of grad_filter is wrong!


---
#Q2 [5Pts]: Implement ReLU and its gradient

**Task**: Implement ReLU operator, i.e., $f(x) = max(x, 0)$, and its gradient operator ${\frac{\partial f}{\partial x}}^{\top} v$ for any given tensor $v$ that is of the same shape as $x$.

**N.B.**: For simplicity, we can assume the input $x$ is of shape $B \times C \times H \times W$ as before.

In [26]:
# vector = np.array([1, -1, 0, 1, 2, -3])
# print(vector.shape)
# for i in range(6):
#   vector[i] = max(0, vector[i])
# print(vector)

In [27]:
# a = np.zeros((3,3))
# a[0, :] = -1
# a[1, :] = 0
# a[2, :] = 2
# print(a)
# new_a = np.maximum(a, 0)
# print(new_a)

In [28]:
## implement the following functions
def func_relu(x): #element-wise...?
    ### Fill in this function ###
    # Args:
    #   x: input, shape B x C x H x W
     #get input shape
  #   B, C, H, W = x.shape
    #tensor_2_numpy_x = x.detach().numpy()
    input2torchmat = torch.zeros((x.shape))
    return torch.tensor(torch.maximum(x, input2torchmat))

    # # Returns:
    # #   y: output, shape B x C x H x W
    # pass
    

def grad_relu(x, y, grad_out):
    ### Fill in this function ###
    # Args:
    #   x: input, shape B x C x H x W
    #   y: output, shape B x C x H x W
    #   grad_out: gradient w.r.t. output y, shape B x D x H x W
    #
    bool = x > 0
    #print(bool.shape)
    grad_x = bool * grad_out

    # Returns:
    #   grad_x: gradient w.r.t. x, shape B x C x H x W
    return grad_x
    #pass


def unit_test_relu(x):    
    x.requires_grad = True
    # print(x.shape)
    # call your implemented "func_relu" function
    y = func_relu(x)

    # ground truth ReLU
    y_gt = F.relu(x)

    # print(y)
    # print(y_gt)
    diff = (y - y_gt).norm()    
    if diff < 1.0e-5:
        print("Your implementation of func_relu is correct!")
    else:
        print("Your implementation of func_relu is wrong!")

    # create a random vector v
    v = torch.randn_like(y)

    # call your implemented "grad_relu" function
    grad_x = grad_relu(x, y, v)

    # compute ground-truth gradients
    grad_x_gt = torch.autograd.grad(y_gt, x, grad_outputs=v, retain_graph=True)[0]    

    diff = (grad_x - grad_x_gt).norm()    
    if diff < 1.0e-5:
        print("Your implementation of grad_relu is correct!")
    else:
        print("Your implementation of grad_relu is wrong!")        

unit_test_relu(torch.randn_like(img))

Your implementation of func_relu is correct!
Your implementation of grad_relu is correct!


<ipython-input-28-027ee0f4018e>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(torch.maximum(x, input2torchmat))


---
#Q3 [20Pts]: Implement Batch-normalization (BN) for convolution and its gradient

Given a batch of input images $x$ with shape $B \times C \times H \times W$, we compute a single mean and a single standard deviation per channel as below, 
\begin{align}
    \mu[c] &= \frac{1}{BHW} \sum_{i=1}^{B} \sum_{m=1}^{H} \sum_{n=1}^{W} x[i, c, m, n] \\
    \sigma^2[c] &= \frac{1}{BHW} \sum_{i=1}^{B} \sum_{m=1}^{H} \sum_{n=1}^{W} (x[i, c, m, n] - \mu[c])^2.
\end{align}
Then we perform BN, $y = f(x, \beta, \gamma)$, as,
\begin{align}
    y[i,c,m,n] &= \gamma[c] \frac{x[i,c,m,n] - \mu[c]}{\sqrt{\sigma^2[c] + \epsilon}} + \beta[c],
\end{align}
where $\gamma$ and $\beta$ are learnable parameters are of shape $C$.
$ϵ$ is a constant.

**Task**: For simplicity, we fix the learnable parameters as $\gamma = 1$ and $\beta = 0$.
Implement BN for convolution and its gradient operators ${\frac{\partial f}{\partial x}}^{\top} v$ for any $v$ that is compatible with the matrix multiplication. 



In [29]:
# C = 3
# m = [0 for i in range(C)]
# print(m)

In [30]:
# a = np.zeros((5,2,3,3))
# a[:,1:2,:,:].shape

In [31]:
## implement the following functions
def func_batch_norm(x, epsilon=1.0e-5):
    ### Fill in this function ###
    # Args:
    #   x: input, shape B x C x H x W
    #   epsilon: constant, scalar
    #
    B, C, H, W = x.shape
    #print(B,C,H,W)
    
    x = x #.detach().numpy()
    # print(x.shape)
    y = torch.zeros((B, C, H, W))

    #mean per channel
    m = [0 for i in range(C)]

    # sd per channel
    sd = [0 for i in range(C)]

    #get mean per channel
    for c in range(C):
      sum = 0
      for b in range(B):
        for h in range(H):
          for w in range(W):
            sum += x[b, c, h, w]
      m[c] = sum / (B*H*W)
    
    # print(m)

    #get std per channel
    for c in range(C):
      sum = 0
      for b in range(B):
        for h in range(H):
          for w in range(W):
            # print(x[b, c, h, w])
            # print(m[c])
            sum += (x[b, c, h, w] - m[c])**2
      sd[c] = sum / (B*H*W)

    # print(sd)

    #perform BN
    for b in range(B):
      for c in range(C):
        for h in range(H):
          for w in range(W):
            y[b,c,h,w] = (x[b,c,h,w] - m[c]) / (torch.sqrt(sd[c]+epsilon))

    return y #torch.tensor(y)

    # # Returns:
    # #   y: output, shape B x C x H x W
    # pass


def grad_batch_norm(x, y, grad_out, epsilon=1.0e-5):
    # ### Fill in this function ###
    # # Args:
    # #   x: input, shape B x C x H x W
    # #   y: output, shape B x C x H x W
    # #   grad_out: gradient w.r.t. output y, shape B x C x H x W
    # #   epsilon: constant, scalar
    # #

    B, C, H, W = x.shape

    # #tensor to numpy
    # x = x.detach().numpy()
    # #y = y.detach().numpy()
    # grad_out = grad_out.detach().numpy()
    # #print(grad_out.shape) # (5,1,28,28)
    # grad_x = np.zeros((B, C, H, W))
    
    # #mean per channel
    # m = [0 for i in range(C)]

    # # sd per channel
    # sd = [0 for i in range(C)]

    # #get mean per channel
    # for c in range(C):
    #   sum = 0
    #   for b in range(B):
    #     for h in range(H):
    #       for w in range(W):
    #         sum += x[b, c, h, w]
    #   m[c] = sum / (B*H*W)
    
    # # print(m)

    # #get std per channel
    # for c in range(C):
    #   sum = 0
    #   for b in range(B):
    #     for h in range(H):
    #       for w in range(W):
    #         # print(x[b, c, h, w])
    #         # print(m[c])
    #         sum += (x[b, c, h, w] - m[c])**2
    #   sd[c] = sum / (B*H*W)

    # #update gradient
    # grad_loss_sd = np.zeros((B,C,H,W))
    # for b in range(B):
    #   for h in range(H):
    #     for w in range(W):
    #       for c in range(C):
    #         for cc in range(C):
    #           grad_loss_sd[b,c:c+1,h,w] += (grad_out[b,cc:cc+1,h,w]*(x[b,cc:cc+1,h,w]-m[c])*(-0.5)*((sd[c]+epsilon)**(-2/3)))
            
    # # for c in range(C):
    # #   for cc in range(C):
    # #     grad_loss_sd[:,c:c+1,:,:] += (grad_out[:,cc:cc+1,:,:]*(x[:,cc:cc+1,:,:]-m[c])*(-0.5)*((sd[c]+epsilon)**(-2/3)))

    # grad_loss_mean = np.zeros((B,C,H,W))
    # tmp1 = np.zeros((B,C,H,W))
    # tmp2 = np.zeros((B,C,H,W))
    # for b in range(B):
    #   for h in range(H):
    #     for w in range(W):
    #       for c in range(C):
    #         for cc in range(C):
    #           #print(grad_out[:,cc,:,:].shape)
    #           tmp1 += grad_out[b,cc:cc+1,h,w]*(-1)*((sd[c]+epsilon)**(-1/2))
    #           tmp2 += -2*(x[b,cc:cc+1,h,w]-m[c])
    #         grad_loss_mean[b,c:c+1,h,w] = tmp1 + grad_loss_sd[b,c:c+1,h,w]*tmp2/ C
    
    # # yj = np.zeros((B,1, H,W))
    # # jeno = np.zeros((B,1, H,W))

    # # for c in range(C):
    # #   print(grad_out[:,c:c+1,:,:].shape)
    # #   yj += grad_out[:,c:c+1,:,:]
    # #   jeno += grad_out[:,c:c+1,:,:]*x[:,c:c+1,:,:]

    # # for c in range(C):
    # #   grad_x[:, c:c+1, :, :] = (m[c]*grad_out[:, c:c+1, :, :] - yj - x[:,c:c+1,:,:]*jeno)*(1/(m[c]*np.sqrt(sd[c]+epsilon)))
    # # for i in range(B): #batchsize
    # #   #ith img from img, grad_img
    # #   #img_1 = x[i] #(C, H, W)
    # #   for h in range(H): #height
    # #     for w in range(W): #width
    # #         for c in range(C): #img's channel  
    # #           #update gradients for the y[i,c,m,n] 
    # #           yj += grad_out[i,c,h,w]
    # #           jeno += x[i, c, h, w] * grad_out[i,c,h,w]
    # #           grad_x[i,c, h, w] = 
    # for b in range(B):
    #   for h in range(H):
    #     for w in range(W):
    #       for c in range(C):
    #         grad_x[b,c:c+1,h,w] = grad_out[b,c:c+1,h,w]*((sd[c]+epsilon)**(-1/2)) + grad_loss_sd[b,c:c+1,h,w]*2*(x[b,c:c+1,h,w]-m[c])/C + grad_loss_mean[b,c:c+1,h,w]*1/C

    # # Returns:
    # #   grad_x: gradient w.r.t. x, shape B x C x H x W
    #return torch.tensor(grad_x)
    return torch.zeros((B,C,H,W))
    #pass 


def unit_test_batch_norm(x):
    x.requires_grad = True
    epsilon = 1e-5

    # call your implemented "func_batch_norm" function
    y = func_batch_norm(x, epsilon=epsilon)

    # ground truth ReLU
    BN_gt = nn.BatchNorm2d(x.shape[1], eps=epsilon, momentum=1.0, affine=False, track_running_stats=False)
    # print(BN_gt)
    y_gt = BN_gt(x)
    # print("y_gt is ", y_gt)
    # print("y is ", y)
    diff = (y - y_gt).norm()  
    # print("diff is ", diff)  
    # print(1.0e-5)
    if diff < 1.0e-5:
        print("Your implementation of func_batch_norm is correct!")
    else:
        print("Your implementation of func_batch_norm is wrong!")

    # create a random vector v
    v = torch.randn_like(y)

    #call your implemented "grad_batch_norm" function
    grad_x = grad_batch_norm(x, y, v, epsilon=epsilon)

    # compute ground-truth gradients
    grad_x_gt = torch.autograd.grad(y_gt, x, grad_outputs=v, retain_graph=True)[0]  #Computes and returns the sum of gradients of outputs with respect to the inputs.

    diff = (grad_x - grad_x_gt).norm()
    #print(diff)
    if diff < 1.0e-5:
        print("Your implementation of grad_batch_norm is correct!")
    else:
        print("Your implementation of grad_batch_norm is wrong!")

unit_test_batch_norm(torch.randn_like(img))

Your implementation of func_batch_norm is correct!
Your implementation of grad_batch_norm is wrong!


---
#Q4 [15Pts]: Implement a simple CNN and back-propagation (BP)

Now we are ready to build a deep CNN and learn it with back-propagation.
In particular, let us build a simple CNN with following architecture:

Conv $→$ BN $→$ ReLU $→$ Conv $→$ BN $→$ ReLU $→$ Linear.

Here, for all layers, the convolutions are the same as before (i.e., kernel size $3 \times 3$, zero-padding, number of filters $D = 2$, and stride 1), the BNs are without learnable $\gamma$ and $\beta$, and the last linear layer would map whatever input dimension to $10$ classes in MNIST. 

**Task**: Implement the above CNN, compute the cross-entropy loss, and compute gradient of the loss w.r.t. filter weights.

**N.B.**: You can use F.cross_entropy provided by PyTorch. But for other operators like Conv, BN, and ReLU and their gradietns, you should use your previous implementations.

In [32]:
## implement the following two functions
def CNN(img, filter_1, filter_2, weight, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    ### Fill in this function ###
    # Args:
    #   img: images, shape B x C x H x W
    #   filter_1: filters at 1st layer, shape D x C x K x K 
    #   filter_2: filters at 2nd layer, shape D x C x K x K
    #   weight: weights of linear readout layer, shape ? x 10
    #   channel_size: number of channels, scalar (C)
    #   num_filters: number of filters, scalar (D)
    #   kernel_size: kernel size, scalar (K)
    #   stride: stride size, scalar
    #   padding: padding size, scalar
    #
    # Returns:
    #   out: logits, shape B x 10
    
    #print("img shape is ", img.shape)
    img = img#.detach().numpy()
    filter_1 = filter_1#.detach().numpy()
    filter_2 = filter_2#.detach().numpy()
    ### 1st layer
    conv1 = conv2d_im2col(img, filter_1, channel_size, num_filters, kernel_size, stride, padding)
    #print("conv1 shape is ", conv1.shape)
    BN1 = func_batch_norm(conv1, epsilon=1.0e-5)
    #print("BN1 shape is ", BN1.shape)
    ReLU1 = func_relu(BN1)
    #print("RELU1 shape is ", ReLU1.shape)
    
    ### 2nd layer
    conv2 = conv2d_im2col(ReLU1, filter_2, channel_size, num_filters, kernel_size, stride, padding)
    #print("conv2 shape is ", conv2.shape)
    BN2 = func_batch_norm(conv2, epsilon=1.0e-5)
    #print("BN2 shape is ", BN2.shape)
    ReLU2 = func_relu(BN2)
    #print("RELU2 shape is ", ReLU2.shape) #RELU2 shape is  torch.Size([5, 2, 28, 28])

    ### linear readout
    weight = weight#.detach().numpy()
    #print("weight shape is ", weight.shape)
    flatten_relu2 = torch.reshape(ReLU2, (B,-1))#ReLU2.reshape(B,-1)
    logits = torch.matmul(flatten_relu2, weight) #np.dot(flatten_relu2, weight) #
    #print("logits shape is ",logits.shape)

    return logits #torch.tensor(logits)
    #pass


def grad_CNN(img, filter_1, filter_2, weight, grad_loss, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    ### Fill in this function ###
    # Args:
    #   img: images, shape B x C x H x W
    #   filter_1: filters at 1st layer, shape D x C x K x K
    #   filter_2: filters at 2nd layer, shape D x C x K x K
    #   weight: weights of linear readout layer, shape ? x 10
    #   grad_loss: gradient of loss w.r.t. logits, shape B x 10 loss/y
    #   channel_size: number of channels, scalar (C)
    #   num_filters: number of filters, scalar (D)
    #   kernel_size: kernel size, scalar (K)
    #   stride: stride size, scalar
    #   padding: padding size, scalar
    #
    # Returns:
    #   grad_filter_1: filters, shape D x C x K x K
    #   grad_filter_2: filters, shape D x C x K x K
    #   grad_weight: weight, shape ? x 10
    
    #CNN
    img = img#.detach().numpy()
    filter_1 = filter_1#.detach().numpy()
    filter_2 = filter_2#.detach().numpy()
    ### 1st layer
    conv1 = conv2d_im2col(img, filter_1, channel_size, num_filters, kernel_size, stride, padding)
    #print("conv1 shape is ", conv1.shape)
    BN1 = func_batch_norm(conv1, epsilon=1.0e-5)
    #print("BN1 shape is ", BN1.shape)
    ReLU1 = func_relu(BN1)
    #print("RELU1 shape is ", ReLU1.shape)
    
    ### 2nd layer
    conv2 = conv2d_im2col(ReLU1, filter_2, channel_size, num_filters, kernel_size, stride, padding)
    #print("conv2 shape is ", conv2.shape)
    BN2 = func_batch_norm(conv2, epsilon=1.0e-5)
    #print("BN2 shape is ", BN2.shape)
    ReLU2 = func_relu(BN2)
    #print("RELU2 shape is ", ReLU2.shape) #RELU2 shape is  torch.Size([5, 2, 28, 28])

    ### linear readout
    weight = weight#.detach().numpy()
    #print("weight shape is ", weight.shape)
    flatten_relu2 = torch.reshape(ReLU2,(B,-1))
    logits = torch.matmul(flatten_relu2, weight) #
    #print("logits shape is ",logits.shape)
    ##

    weight1shape, weight2shape = weight.shape
    grad_filter_1 = torch.zeros((D,C,K,K))
    grad_filter_2 = torch.zeros((D,C,K,K))
    grad_weight = torch.zeros((weight1shape, weight2shape))
    
    # y = CNN(img, filter_1, filter_2, weight, channel_size=channel_size, num_filters=num_filters, kernel_size=kernel_size, stride=stride, padding=padding)
    # loss = F.cross_entropy(y, label).mean()

    ### linear readout (grad_weight)
    # print("flatten_relu2 shape is ", flatten_relu2.shape)
    # print("grad_loss shape is ", grad_loss.shape)
    # print(torch.transpose(flatten_relu2, 0,1).shape)
    grad_weight = torch.matmul(torch.transpose(flatten_relu2,0,1),grad_loss)
    print(grad_weight.shape)
    ### 2nd layer (grad_filter2)


    ### 1st layer (grad_filter1)

    

    return  torch.tensor(grad_filter_1),  torch.tensor(grad_filter_2),  torch.tensor(grad_weight)
    
    #pass


def unit_test_CNN(img, label, filter_1, filter_2, weight, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    # call your implemented "CNN"
    img.requires_grad_()
    filter_1.requires_grad_()
    filter_2.requires_grad_()
    weight.requires_grad_()
    y = CNN(img, filter_1, filter_2, weight, channel_size=channel_size, num_filters=num_filters, kernel_size=kernel_size, stride=stride, padding=padding)
    #print("y shape is", y.shape) #remove before submit. It should be (10, )
    y.requires_grad_()

    # compute loss function
    #print("label is", label)
    loss = F.cross_entropy(y, label).mean()
    loss.requires_grad_()

    # compute gradient of loss w.r.t. logits
    grad_loss = torch.autograd.grad(loss, y, retain_graph=True)[0]

    # call your implemented "grad_batch_norm" function
    grad_filter_1, grad_filter_2, grad_weight = grad_CNN(img, filter_1, filter_2, weight, grad_loss, channel_size=channel_size, num_filters=num_filters, kernel_size=kernel_size, stride=stride, padding=padding)

    #print(grad_weight.shape)
    # compute ground-truth gradients
    grad_filter_1_gt = torch.autograd.grad(loss, filter_1, retain_graph=True)[0]
    grad_filter_2_gt = torch.autograd.grad(loss, filter_2, retain_graph=True)[0]
    grad_weight_gt = torch.autograd.grad(loss, weight, retain_graph=True)[0]

    diff = (grad_filter_1 - grad_filter_1_gt).norm()
    if diff < 1.0e-5:
        print("Your implementation of grad_filter_1 is correct!")
    else:
        print("Your implementation of grad_filter_1 is wrong!")

    diff = (grad_filter_2 - grad_filter_2_gt).norm()
    if diff < 1.0e-5:
        print("Your implementation of grad_filter_2 is correct!")
    else:
        print("Your implementation of grad_filter_2 is wrong!")

    diff = (grad_weight - grad_weight_gt).norm()
    if diff < 1.0e-5:
        print("Your implementation of grad_weight is correct!")
    else:
        print("Your implementation of grad_weight is wrong!")                


filter_1 = torch.randn(D, C, K, K) # filter shape: D x C x K x K
filter_2 = torch.randn(D, D, K, K) # filter shape: D x C x K x K

### compute the correct shape and then replace None with it ###
weight = torch.randn(2*28*28, 10) # weight of the last linear layer

unit_test_CNN(img, label, filter_1, filter_2, weight, channel_size=C, num_filters=D, kernel_size=K, stride=1, padding=P)

<ipython-input-28-027ee0f4018e>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(torch.maximum(x, input2torchmat))


torch.Size([1568, 10])


<ipython-input-32-48534e616943>:117: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return  torch.tensor(grad_filter_1),  torch.tensor(grad_filter_2),  torch.tensor(grad_weight)


RuntimeError: ignored